Продолжаем знакомство с моделями МО в области обучения с учителем. На этот раз поговорим о задаче **классификации**. Вспомним, где находится классификация на нашей карте машинного обучения:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-3_2 (1).png)

Вначале мы снова обратимся к классу линейных моделей и рассмотрим **логистическую регрессию**.

↓

Затем поговорим о **деревьях решений** для задачи классификации и научимся строить из этих деревьев целый лес.

**Цели данного модуля:**

* Познакомиться с принципами работы модели логистической регрессии для решения задачи классификации. 
* Рассмотреть метрики классификации и научиться оценивать качество моделей, решающих данную задачу.
* Узнать принципы построения деревьев решений.
* Изучить основы ансамблевых моделей типа бэггинг на примере случайного леса.
* Научиться применять деревья решений и случайные леса для решения задачи классификации.

`✍ Ранее мы обсуждали модель линейной регрессии, которая предназначена для решения задачи регрессии. Теперь нам предстоит разобраться с тем, как преобразовать данную модель, чтобы она решала задачу классификации.`

Для начала вспомним, что такое классификация.

**Задача классификации (classification)** *— задача, в которой мы пытаемся предсказать класс объекта на основе признаков в наборе данных. То есть задача сводится к предсказанию целевого признака, который является **категориальным**.*

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-3_9.png)

Когда классов, которые мы хотим предсказать, только два, классификация называется **бинарной**. Например, мы можем предсказать, болен ли пациент раком, является ли изображение человеческим лицом, является ли письмо спамом и т. д.

Когда классов, которые мы хотим предсказать, более двух, классификация называется **мультиклассовой (многоклассовой)**. Например, предсказание модели самолёта по радиолокационным снимкам, классификация животных на фотографиях, определение языка, на котором говорит пользователь, разделение писем на группы.

    → Для простоты мы пока разберёмся с бинарной классификацией, а в следующем юните обобщим результат на мультиклассовую.

Что вообще означает «решить задачу классификации»? Это значит построить разделяющую поверхность в пространстве признаков, которая делит пространство на части, каждая из которых соответствует определённому классу. 

Ниже представлены примеры разделяющих поверхностей, которые производят бинарную классификацию. Красным и синим цветом обозначены классы, зелёным — собственно поверхность, которая делит пространство признаков на две части. В каждой из этих частей находятся только наблюдения определённого класса.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_1.png)

*Модели, которые решают задачу классификации, называются **классификаторами (classifier)**.*

Если взять в качестве разделяющей поверхности некоторую плоскость (ровная поверхность на первом рисунке), то мы получаем модель логистической регрессии, которая тесно связана с рассмотренной нами ранее линейной регрессией.

Давайте для начала вспомним, как выглядит уравнение модели линейной регрессии в общем случае:

![](./image/2024-08-01_20-59-15.png)

В общем случае это уравнение гиперплоскости, которая стремится приблизить зависимость целевой переменной от $m$ факторов.

* Когда фактор всего один, уравнение задаёт прямую:

$y^` = w_0 + w_1x$

* Когда факторов два, уравнение задаёт плоскость:

$y^`=w_0+w_1x_1+w_2x_2$

→ Но всё это работает только в том случае, когда целевой признак $y$, который мы хотим предсказать, является числовым, например цена, вес, время аренды и т. д.

Что же делать с этой моделью, когда целевой признак $y$ является категориальным? Например, является письмо спамом или обычным письмом?

Можно предположить, что, раз у нас есть две категории, мы можем обозначить категории за $y=1$ (Спам) и $y=0$ (Не спам) и обучить линейную регрессию предсказывать 0 и 1.

Но результат будет очень плохим. Выглядеть это будет примерно так:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_2.png)

Для больших значений $x$ прямая будет выдавать значения больше 1, а для очень маленьких — меньше 0. Что это значит? Непонятно. Непонятно и то, что делать со значениями в диапазоне от 0 до 1. Да, можно относить значения на прямой выше 0.5 к классу 1, а меньше либо равным 0.5 — к классу 0, но это всё «костыли».

`Идея! Давайте переведём задачу классификации в задачу регрессии. Вместо предсказания класса будем предсказывать вероятность принадлежности к этому классу.` 

Модель должна выдавать некоторую вероятность $P$, которая будет определять, принадлежит ли данный объект к классу 1: например, вероятность того, что письмо является спамом. При этом вероятность того, что письмо является обычным письмом (класс 0), определяется как $Q=1-P$.  

Когда модель будет обучена на предсказание вероятности, мы зададим некоторый порог вероятности. Если предсказанная вероятность будет выше этого порога, мы определим объект к классу 1, а если ниже — к классу 0.

Например, стандартный порог равен 0.5. То есть если вероятность $P>0.5$, мы будем считать письмо спамом, а если $P<=0.5$ — обычным информативным письмом.

В итоге мы добьёмся того, что будем предсказывать не дискретный категориальный, а непрерывный числовой признак, который лежит в диапазоне [0, 1]. А это уже знакомая нам задача регрессии.

    → Однако остался главный вопрос: как научить модель предсказывать вероятности, ведь они должны лежать строго в диапазоне от 0 до 1, а предсказания линейной регрессии лежат в диапазоне от $-infinity$ до $+infinity$? 

Тут-то мы и приходим к модели логистической регрессии — **регрессии вероятностей**.

### <center> ОБЩЕЕ ПРЕДСТАВЛЕНИЕ О ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

**Логистическая регрессия (Logistic Regression)** *— одна из простейших моделей для решения задачи классификации. Несмотря на простоту, модель входит в топ часто используемых алгоритмов классификации в Data Science.*

В основе логистической регрессии лежит логистическая функция (*logistic function*) $sigma(z)$ — отсюда и название модели. Однако более распространённое название этой функции — **сигмόида (sigmoid)**. Записывается она следующим образом:

![](./image/2024-08-01_21-07-45.png)

Примечание. Здесь $e$ — экспонента или число Эйлера. Это число является бесконечным, а его значение обычно принимают равным $2.718...$.

А вот график зависимости сигмоиды от аргумента $z$:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_3.png)

**В чём преимущество этой функции?**

*У сигмоиды есть два очень важных для нас свойства:*

* Значения сигмоиды $sigma(z)$ лежат в диапазоне от 0 до 1 при любых значения аргумента $z$: какой бы $z$ вы ни подставили, число меньше 0 или больше 1 вы не получите.
* Сигмоида выдаёт значения $sigma(z)>0.5$ при её аргументе $x>0$, $sigma(z)<0.5$ — при $z<0 и $sigma(z)=0.5$ — при $z=0$.

Это ведь и есть свойства вероятности! Выходом сигмоиды является число от 0 до 1, которое можно интерпретировать как вероятность принадлежности к классу 1. Её мы и пытаемся предсказать.

**Основная идея** модели логистической регрессии: возьмём модель линейной регрессии (обозначим её выход за $z$)

![](./image/2024-08-01_21-17-21.png)

и подставим выход модели $z$ в функцию сигмоиды, чтобы получить искомые оценки вероятности (в математике принято писать оценочные величины с «шапкой» наверху, а истинные значения — без «шапки», это чистая формальность):

![](./image/2024-08-01_21-18-54.png)

**Примечание.** *Далее в модуле мы будем называть оценки вероятности $P^`$ просто вероятностью, но только для краткости. Это не значит, что эти оценки являются истинными вероятностями принадлежности к каждому из классов (их нельзя сосчитать, так как для этого нужна выборка бесконечного объёма). Если вы употребляете термин «вероятности» на собеседованиях, обязательно предварительно укажите, что вы подразумеваете оценку вероятности.*

Обучать будем всё в совокупности, пытаясь получить наилучшую оценку вероятности $P^`$. Если вероятность $P^`>0.5$, относим объект к классу 1, а если $P^`<=0.5$, относим объект к классу 0. 

Математически это записывается следующей формулой:

![](./image/2024-08-01_21-21-56.png)

**Примечание.** *В данном выражении $I[P^`]$ называется **индикаторной функцией**. Она возвращает 1, если её значение больше какого-то порога, и 0 — в противном случае. Математики часто записывают просто квадратные скобки, опуская символ $I$: $[P^`]$.*

**Чего мы добились таким преобразованием?**

Если мы обучим модель, то есть подберём  коэффициенты $w_0, w_1, w_2, ..., w_m$ (как их найти, обсудим чуть позже) таким образом, что для объектов класса 1 модель линейной регрессии начнёт выдавать положительное число, а для класса 0 — выдавать отрицательное число, то тогда, подставив предсказание линейной регрессии $z$ в сигмоиду, мы сможем получать вероятности принадлежности к каждому из классов в диапазоне от 0 до 1.

Далее по порогу вероятности мы сможем определять, к какому классу принадлежит объект.

Это и есть наша цель. Мы свели задачу классификации к задаче регрессии для предсказания вероятностей. 

Для бинарной классификации описанное выше будет выглядеть следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_4.png)

![](./image/2024-08-01_21-26-21.png)

**Разберёмся с геометрией**

Возьмём частный случай, когда класс объекта зависит от двух признаков — $x_1$ и $x_2$.

![](./image/2024-08-01_21-31-25.png)

![](./image/2024-08-01_21-32-32.png)

**В чём математический секрет?**

Математически подстановка в уравнение плоскости точки, которая не принадлежит ей (находится ниже или выше), означает вычисление расстояния от этой точки до плоскости.

* Если точка находится ниже плоскости, расстояние будет отрицательным ($z<0$).
* Если точка находится выше плоскости, расстояние будет положительным ($z>0$).
* Если точка находится на самой плоскости, $z=0$.

Мы знаем, что подстановка отрицательных чисел в сигмоиду приведёт к вероятности $P^`<0.5$, а постановка положительных — к вероятности $P^`>0.5$. 

*Таким образом, ключевым моментом в предсказании логистической регрессии является расстояние от точки до разделяющей плоскости в пространстве факторов. Это расстояние в литературе часто называется* **отступом (margin)**. 

В этом и состоит секрет работы логистической регрессии.

*Чем больше расстояние от точки, находящейся выше разделяющей плоскости, до самой плоскости, тем больше оценка вероятности принадлежности к классу 1.*

Попробуйте подставить различные координаты точек в модель логистической регрессии и убедитесь в этом.

Можно построить тепловую карту, которая показывает, чему равны вероятности в каждой точке пространства:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_7.png)

`На рисунке точки, которые относятся к классу непоступивших абитуриентов, лежащие ниже разделяющей плоскости, находятся в красной зоне. Чем насыщеннее красный цвет, тем ниже вероятность того, что абитуриент поступит в аспирантуру. И наоборот, точки, которые относятся к классу поступивших абитуриентов, лежащие выше разделяющей плоскости, находятся в синей зоне. Чем насыщеннее синий цвет, тем выше вероятность того, что абитуриент поступит в аспирантуру.`

Для случая зависимости целевого признака $y$ от трёх факторов $x_1$, $x_2$ и $x_3$, например от баллов за два экзамена и рейтинга университета, из которого выпустился абитуриент, выражение для $z$ будет иметь вид:

$z=w_0+w_1x_1+w_2x_2+w_3x_3$

Уравнение задаёт плоскость в четырёхмерном пространстве. Но если вспомнить, что $y$ — категориальный признак и классы можно обозначить цветом, то получится перейти в трёхмерное пространство. Разделяющая плоскость будет выглядеть следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_8.png)

В общем случае, когда у нас есть зависимость от $m$ факторов, линейное выражение, находящееся под сигмоидой, будет обозначать **разделяющую гиперплоскость**.

![](./image/2024-08-01_21-39-41.png)

### <center> ПОИСК ПАРАМЕТРОВ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

Итак, мы разобрались с тем, как выглядит модель логистической регрессии и что она означает в геометрическом смысле.

Но остался один главный вопрос: как найти такие коэффициенты $w=(w_0, w_1, w_2, ..., w_m)$, чтобы гиперплоскость разделяла пространство наилучшим образом?

Вновь обратимся к нашей схеме минимизации эмпирического риска:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-2_1.png)

Можно предположить, что стоит использовать метод наименьших квадратов. Введём функцию ошибки — средний квадрат разности MSE между истинными классами $y$ и предсказанными классами $y^`$ и попытаемся его минимизировать.

Сразу можно достоверно предсказать, что результат такого решения будет плохим, поэтому воздержимся от его использования.

Здесь нужен другой подход. Это **метод максимального правдоподобия (Maximum Likelihood Estimation — MLE)**. 

**Правдоподобие** *— это оценка того, насколько вероятно получить истинное значение целевой переменной  при данных  и параметрах $w$.*

Данный метод позволяет получить функцию правдоподобия.

**Цель метода** *— найти такие параметры $w=(w_0, w_1, w_2, ..., w_m)$, в которых наблюдается максимум функции правдоподобия.*

А мы пока что опустим математические детали метода и приведём только конечную формулу:

![](./image/2024-08-01_21-45-58.png)

Не пугайтесь. Давайте разберёмся, что есть что и как работает эта функция.

* $n$ — количество наблюдений.
* $y_i$ — это истинный класс (1 или 0) для -ого объекта из набора данных.
* $P^`=sigma(z_i)$ — предсказанная с помощью логистической регрессии вероятность принадлежности к классу 1 для $i$-ого объекта из набора данных.
* $z_i$ — результат подстановки -ого объекта из набора данных в уравнение разделяющей плоскости $z_i=w^`*x_i^`$.
* $log$ — логарифм (обычно используется натуральный логарифм по основанию $e-ln$).

![](./image/2024-08-01_21-54-39.png)

Такие расчёты можно производить для любых значений параметров, меняется только оценка вероятности $P_i^`$.

**Примечание.** *К сожалению, функция likelihood не имеет интерпретации, то есть нельзя сказать, что значит число -2.34 в контексте правдоподобия.*

Цель — найти такие параметры, при которых наблюдается максимум этой функции.

Теперь пора снова применить магию математики, чтобы привести задачу к привычному нам формату минимизации эмпирического риска. По правилам оптимизации, если поставить перед функцией минус, то задача оптимизации меняется на противоположную: был поиск максимума — станет поиском минимума.

Таким образом мы получим функцию потерь $L(w)$, которая носит название **«функция логистических потерь»**, или **logloss**. Также часто можно встретить название **кросс-энтропия**, или **cross-entropy loss**:

![](./image/2024-08-01_21-56-49.png)

Вот эту функцию мы и будем минимизировать в рамках поиска параметров логистической регрессии. Мы должны найти такие параметры разделяющей плоскости $w$, при которых наблюдается минимум logloss.

Знакомая задача? Всё то же самое, что и с линейной регрессией, только функция ошибки другая.

    → К сожалению, для такой функции потерь аналитическое решение оптимизационной задачи найти не получится: при расчётах получается, что его попросту не существует.

Но мы помним, что, помимо аналитических решений, есть и численные.

Например, для поиска параметров можно использовать знакомый нам градиентный спуск. Вспомним, как выглядит итерационная формула данного метода:

![](./image/2024-08-01_21-58-06.png)

Повторим её смысл: новое значение параметров $w^{(k+1)}$ получается путём сдвига текущих $w^{(k)}$ в сторону вектора антиградиента $Grad L(w^{(k)})$, умноженного на темп обучения $nu$.

    Мы уже знаем, что для того, чтобы повысить шанс пройти мимо локальных минимумов функции потерь, используется не сам градиентный спуск, а его модификации: например, можно использовать уже знакомый нам стохастический градиентный спуск (SGD).

`Помним, что применение градиентного спуска требует предварительного масштабирования данных (стандартизации/нормализации). В реализации логистической регрессии в sklearn предусмотрено ещё несколько методов оптимизации, для которых масштабирование не обязательно. О них мы упомянем в практической части модуля.`

Во избежание переобучения модели в функцию потерь логистической регрессии традиционно добавляется **регуляризация**. В реализации логистической регрессии в *sklearn* она немного отличается от той, что мы видели ранее для линейной регрессии.

При L1-регуляризации мы добавляем в функцию потерь $L(w)$ штраф из суммы модулей параметров, а саму функцию logloss умножаем на коэффициент $C$:

![](./image/2024-08-01_22-10-48.png)

Значение коэффициента $C$ — коэффициент, обратный коэффициенту регуляризации. Чем **больше** $C$, тем **меньше** «сила» регуляризации.

Предлагаем вам посмотреть на то, как будет меняться форма сигмоиды, разделяющей плоскости при минимизации функции потерь logloss (она обозначена как cross-entropy в виде концентрических кругов — вид сверху), с помощью обычного градиентного спуска (не стохастического) в виде анимации.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_12.gif)
![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_13.gif)
![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_14.gif)

Не волнуйтесь, все громоздкие формулы уже реализованы в классических библиотеках, таких как sklearn. Но нам важно понимать принцип того, что происходит «под капотом», чтобы верно интерпретировать результат и по возможности управлять им.

### <center> ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ В SKLEARN

    Мы будем работать со знакомым вам по модулю «Очистка данных» набором данных о диабете, первоначально полученным в Национальном институте диабета, болезней органов пищеварения и почек.

    Наша цель будет состоять в том, чтобы диагностически предсказать, есть ли у пациента диабет. На выбор экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года из индейского племени Пима.

В модуле по очистке данных вы уже производили очистку этого набора данных:

* удалили дубликаты,
* удалили неинформативный признак Gender,
* обработали «скрытые» пропуски в данных,
* избавились от потенциальных выбросов.

#### Результат представлен в ./Логистическая_регрессия.ipynb

    ✍ Мы научились обучать модель логистической регрессии, чтобы решать с её помощью задачу классификации. Теперь мы познакомимся с метриками классификации, чтобы научиться оценивать качество модели.

Однако прежде чем мы перейдём к этим метрикам, нам очень важно освежить в памяти некоторую терминологию из статистики, а именно вспомнить, что такое ошибки I и II рода.

### <center>ОШИБКИ I И II РОДА С ТОЧКИ ЗРЕНИЯ КЛАССИФИКАЦИИ

Давайте рассмотрим предсказания алгоритма $y_i^`$ на конкретном объекте $x_i$ под номером из данных $i$ с точки зрения статистических гипотез.

Будем считать класс 1 (диабет есть) положительным исходом (Positive), а класс 0 (диабета нет) — отрицательным (Negative).

**Примечание.** *На первый взгляд такая терминология может показаться не совсем этичной, так как наличие диабета — это всё-таки отрицательный сценарий для пациента. Однако для унификации терминологии в машинном обучении в большинстве задач объекты класса 1 считаются объектами с наличием некоторого эффекта (болезнь есть / задолженность погашена / клиент ушёл / устройство отказало и т. д.), а объекты класса 0 — объектами с отсутствием этого эффекта (болезни нет / задолженность не погашена / клиент не ушёл / устройство работает без отказов и т. д.).*

Пусть у нас есть некоторый пациент $x_i$, и мы хотим понять, болен ли он диабетом. С точки зрения задачи классификации мы хотим предсказать истинный класс ($y_i$) пациента.

Нулевая гипотеза будет состоять в отсутствии эффекта (пациент не болен диабетом), то есть , а альтернативная — в его наличии (пациент болен диабетом) $y_i=0$, то есть $y_i=1$. В терминах статистических гипотез это будет записано так:

* $H_0$: Пациент $x_i$ не болеет диабетом $y_i=0$.
* $H_1$: Пациент $x_i$ болеет диабетом $y_i=1$.
Тогда у нас есть два случая, в которых мы можем допустить ошибку:

Ошибка I (первого) рода ($alpfa$-ошибка): отклонение нулевой гипотезы, когда она на самом деле верна, или **ложноположительный результат**. То есть мы предсказали, что пациент болен диабетом, хотя это не так.
Ошибка II (второго) рода ($betta$-ошибка): принятие нулевой гипотезы, когда она на самом деле ложна, или **ложноотрицательный результат**. То есть мы предсказали, что пациент здоров, хотя на самом деле он болен диабетом.

**Примечание.** *Как вы можете понять, в диагностических задачах для нас критичнее ошибка II рода. Последствия будут более серьёзными, если мы примем больного пациента за здорового, чем если мы примем здорового за больного. Нам важно охватить всех потенциально больных пациентов, чтобы сделать дополнительный анализ и удостовериться в результате.*

    Отлично, мы освежили в памяти ошибки I и II рода — это поможет нам понять суть метрик классификации. Давайте перейдём к ним.

### <center>МЕТРИКИ КЛАССИФИКАЦИИ

Будем рассматривать метрики для задачи классификации на следующем примере.

Мы случайным образом выбрали десять пациентов из нашей таблицы и моделью log_reg_full предсказали для них ответы:

![](./image/ML_3_1.png)

Все метрики, которые мы рассмотрим, основаны на матрице ошибок. С неё мы и начнём наш разбор.

1. **Матрица ошибок (confusion matrix)** *показывает все возможные исходы совпадения и несовпадения предсказания модели с действительностью. Используется для расчёта других метрик.*

Допустим, что у нас есть два класса и алгоритм, предсказывающий принадлежность каждого объекта к одному из классов. Тогда каждая ячейка матрицы ошибок соответствует количеству объектов, попавших в одну из следующих четырёх категорий:

* **Истинно положительные (True Positive, TP)** — это объекты, обозначенные моделью как класс 1 ($y^`=1$) и действительно принадлежащие к классу 1 ($y=1$).
* **Ложноположительные (False Positive, FP)** — это объекты, обозначенные моделью как класс 1 ($y^`=1$), но в действительности принадлежащие к классу 0 ($y=0$). То есть это объекты, для которых модель совершила ошибку I рода.
* **Истинно отрицательные (True Negative, TN)** — это объекты, обозначенные моделью как класс 0 ($y^`=0$) и действительно принадлежащие к классу 0 ($y=0$).
* **Ложноотрицательные (False Negative, FN)** — это объекты, обозначенные моделью как класс 0 ($y^`=0$), но в действительности принадлежащие к классу 1 ($y=1$). То есть это объекты, для которых модель совершила ошибку II рода.

Общий вид матрицы ошибок будет следующим:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_1.png)

Давайте посмотрим, как будет выглядеть матрица ошибок для нашего примера предсказаний модели *log_reg_full*:

![](./image/ML_3_2.png)

Для большей наглядности представим следующую ситуацию: у нас есть множество наблюдений двух классов (класс 1 — круги, класс 0 — квадраты). Пусть мы нарисовали некоторый круг и условились, что все объекты, лежащие в этом круге, мы будем считать классом 1, а объекты вне круга — классом 0. Тогда мы получим следующую картину:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_2.png)

Таким образом, мы получили четыре группы объектов:

* Объекты, попавшие в круг, для которых мы предсказали класс 1, и эти точки действительно относятся к классу 1 — *True Positive (TP)*.

* Объекты, попавшие в круг, для которых мы предсказали класс 1, но эти точки относятся к классу 0 — *False Positive (FP)*.

* Объекты за пределами круга, для которых мы предсказали класс 0, и эти точки действительно относятся к классу 0 — *True Negative (TN)*.

* Объекты за пределами круга, для которых мы предсказали класс 0, но эти точки относятся к классу 1 — *False Negative (FN)*.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_3.png)

Формально матрица ошибок не является метрикой, но на её основе составляются сами метрики классификации. Давайте перейдём к ним.

2. **Accuracy (достоверность/аккуратность)** *— доля правильных ответов модели среди всех ответов. Правильные ответы — это истинно положительные (True Positive) и истинно отрицательные ответы (True Negative):*

![](./image/ML_3_3.png)

**Примечание.** Нередко в русской литературе вы можете встретить перевод метрики *accuracy* как «точность», однако так же на русский язык переводится метрика *precision*, о которой мы поговорим далее. Поэтому, если вы используете термин «точность», старайтесь указывать, о какой именно метрике (*accuracy* или *precision*) идёт речь.

В виде диаграммы соотношение количества объектов, классы которых мы угадали и общего количества объектов записывается в следующем виде:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_4.png)

**Интерпретация:** *как много (в долях) модель угадала ответов.*

Метрика изменяется в диапазоне от 0 до 1. Чем ближе значение к 1, тем больше ответов модель «угадала». 

Рассчитаем accuracy для нашего примера:

![](./image/ML_3_4.png)

Итак, наша accuracy равна 0.5, то есть модель сделала верное предсказание для 50 % пациентов из выборки.

`Accuracy — самая простая и самая понятная метрика классификации, но у неё есть один существенный недостаток. Она бесполезна, если классы сильно несбалансированы. Продемонстрируем это следующим примером.`

    Допустим, мы хотим оценить работу спам-фильтра электронной почты. Спам-письма обозначены как Positive (1), а не спам — как Negative (0).

У нас есть 110 писем. Из них:

* 100 писем — не спам,
* 10 писем — спам.

Мы построили некоторый классификатор. Согласно результатам классификатора, из 100 писем класса «не спам» мы верно определили (отнесли к классу 0) 90, а оставшиеся 10 отнесли к классу «спам» (классу 1). То есть True Negative = 90, а False Positive = 10. Из 10 спам-писем мы верно определили 5 (отнесли к классу 1), а остальные 5 отнесли к классу «не спам» (классу 0). То есть True Positive = 5, а False Negative = 5.

Матрица ошибок будет иметь вид:

![](./image/ML_3_5.png)

Тогда *accuracy*:

![](./image/ML_3_6.png)

Однако представим, что мы построили классификатор, который просто предсказывает все письма как «не спам», то есть True Negative = 100, False Negative = 10, True Positive = 0, False Positive = 0.

Матрица ошибок будет иметь вид:

![](./image/ML_3_7.png)

Тогда *accuracy* будет равна:

![](./image/ML_3_8.png)

    Метрика выросла, однако наша «модель» не обладает никакой предсказательной силой, так как изначально мы хотели определять письма со спамом. Преодолеть это нам поможет переход с общей для всех классов метрики к отдельным показателям качества классов, а именно к метрикам precision, recall и -мера.

3. **Precision (точность), или PPV (Positive Predictive Value)** *— это доля объектов, названных классификатором положительными и при этом действительно являющихся таковыми, по отношению ко всем названным положительными объектам.*

![](./image/ML_3_9.png)

В виде диаграммы соотношение количества объектов класса 1, которые мы угадали и количества объектов, которые мы приняли за класс 1, записывается следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_5.png)

Метрика также изменяется от 0 до 1. 

**Интерпретация:** способность отделить класс 1 от класса 0. Чем больше *precision*, тем меньше ложных попаданий. **То есть чем ближе *precision* к 1, тем меньше вероятность модели допустить ошибку I рода.**

Именно *precision* не позволяет записать все ответы в один класс, так как в таком случае резко возрастает значение *False Positive* и метрика снижается.

Рассчитаем *precision* для нашего примера:

![](./image/ML_3_10.png)

Таким образом, количество названных моделью больных диабетом и при этом действительно являющихся больными составляет 67 % от всех пациентов.

Precision нужен в задачах, где от нас требуется минимум ложных срабатываний. Чем выше «цена» ложноположительного результата, тем выше должен быть *precision*.

Например, по камерам видеонаблюдения мы автоматически выявляем признаки драки на улицах и отправляем наряд полиции для урегулирования конфликта. Однако штат сотрудников сильно ограничен, реагировать на любой признак конфликта мы не можем, поэтому мы хотим уменьшить количество ложных вызовов. В такой ситуации мы выберем такую модель, у которой наибольший *precision*.

В предельном случае (когда *precision* равен 1) у модели отсутствуют ложноположительные срабатывания.

**Примечание.** *Важно понимать, что данный вывод справедлив только для выборки, на которой мы оцениваем метрику, то есть это не означает, что модель вовсе не может допустить ложноположительных результатов. Однако чем больше выборка, на которой мы тестируем алгоритм, тем ближе к истине будет данный вывод.*

4. **Recall (полнота), или TPR (True Positive Rate)** *— это доля объектов, названных классификатором положительными и при этом действительно являющихся таковыми, по отношению ко всем объектам положительного класса.*

![](./image/ML_3_11.png)

Диаграмма:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_6.png)

Метрика изменяется от 0 до 1.

**Интерпретация:** способность модели обнаруживать класс 1 вообще, то есть охват класса 1. Заметьте, что метрика зависит от количества ложноотрицательных срабатываний. **То есть чем ближе recall к 1, тем меньше вероятность модели допустить ошибку II рода.**

Рассчитаем *recall* для нашего примера:

![](./image/ML_3_12.png)

Итак, процент пациентов, которых модель определила к классу больных диабетом, среди всех действительно больных диабетом составляет 57 %.

*Recall* очень хорошо себя показывает в задачах, где важно найти как можно больше объектов, принадлежащих к классу 1.

Например, это различные задачи, в которых мы пытаемся определить наличие эффекта, который может привести к серьёзным последствиям. Это могут быть те же диагностические задачи или задачи, в которых мы прогнозируем вероятность отказа устройства, от работы которого зависят человеческие жизни.

Предельный случай (когда *recall* равен 1) означает, что модель нашла все объекты класса 1, например всех действительно больных пациентов. Однако метрика ничего не скажет о том, с какой точностью мы это сделали.

**Примечание.** *Важно понимать, что данный вывод справедлив только для выборки, на которой мы оцениваем метрику, то есть это не означает, что модель вовсе не может допустить ложноотрицательных исходов. Однако чем больше выборка, на которой мы тестируем алгоритм, тем данный вывод будет ближе к истине.*

Метрики *precision* и *recall* не зависят от сбалансированности классов и в совокупности дают довольно исчерпывающее представление о классификаторе. Однако на практике часто бывает так, **что увеличение одной из метрик может привести к уменьшению другой.**

Концентрация только на одной метрике (*precision* или *recall*) без учёта второй — сомнительная идея.

В битве за максимум *precision* для класса 1 побеждает модель, которая всегда будет говорить «нет». У неё вообще не будет ложноположительных срабатываний.

В битве за максимум *recall* для класса 1 побеждает модель, которая всегда будет говорить «да». Она охватит все наблюдения класса 1, и у неё не будет ложноотрицательных срабатываний. 

В реальности необходимо балансировать между двумя этими метриками.

    Классическим примером является задача определения оттока клиентов.

    Очевидно, что мы хотим найти как можно больше клиентов, которые потенциально могут уйти от нас. Чтобы повысить их лояльность, мы планируем использовать ресурсы колл-центра. Однако они ограничены и мы не можем звонить всем клиентам. Определив стратегию и ресурс для удержания клиентов, мы можем подобрать нужные пороги по precision и recall. Например, можно сосредоточиться на удержании только высокодоходных клиентов или тех, кто уйдёт с большей вероятностью. 

5. В таком случае нам подойдёт следующая метрика.

*$F_{betta}$ **(F-мера)** — это **взвешенное среднее гармоническое** между *precision* и *recall*:*

![](./image/ML_3_13.png)

*где $betta$ — это вес *precision* в метрике: чем больше $betta$, тем больше вклад.*

В частном случае, когда $betta=1$, мы получаем равный вклад для *precision* и *recall*, а формула будет выражать простое среднее гармоническое, или метрику $F_1$ ($F_1$-мера):

![](./image/ML_3_14.png)

Рассчитаем метрику $F_1$ для нашего примера:

![](./image/ML_3_15.png)

**В чём преимущество $F_1$-меры?**

*Метрика равна своему максимуму (1), если и precision, и recall равны 1 (то есть когда отсутствуют как ложноположительные, так и ложноотрицательные срабатывания). Но если хотя бы одна из метрик будет близка к 0, то и  будет близка к 0.*

Несмотря на отсутствие бизнес-интерпретации, метрика $F_1$ является довольно распространённой и используется в задачах, где необходимо выбрать модель, которая балансирует между precision и recall.

    Например, если цена дополнительной диагностики заболевания очень высока, то есть ложных срабатываний должно быть минимум, но при этом нам важно охватить как можно больше больных пациентов. 

**Примечание.** *Ещё одно небольшое, но очень важное замечание: все суждения, которые мы привели по отношению к precision, recall и $F$-мере, относятся только к классу 1, так как эти метрики по умолчанию считаются для класса 1. Для решения большинства задач знания о значении этих метрик для класса 1 более чем достаточно, так как обычно нас интересует именно наличие некоторого эффекта.*

*Однако если вам по каким-то причинам необходимо рассчитать precision, recall и $F$-меру для класса 0, для этого достаточно сделать перекодировку классов — поменять их обозначения местами или (при расчёте метрик с помощью библиотеки sklearn) изменить значение специального параметра pos_label на 0.*

Давайте обобщим всё вышесказанное в виде таблицы:

![](./image/ML_3_16.png)

### <center>РАСЧЁТ МЕТРИК НА PYTHON

Код и результаты находятся в `Логическая_регресия.ipynb/Метрики классификации`

### <center>ДОСТОИНСТВА И НЕДОСТАТКИ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ 

Давайте обобщим всё вышесказанное и приведём достоинства и недостатки логистической регрессии.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@happy-icon.png)

* Простой, интерпретируемый, но в то же время эффективный алгоритм. Благодаря этому он очень популярен в мире машинного обучения.
* Поиск параметров линейный или квадратичный (в зависимости от метода оптимизации), то есть ресурсозатратность алгоритма очень низкая.
* Не требует подбора внешних параметров (гиперпараметров), так как практически не зависит от них.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@sad-icon.png)

* Алгоритм работает хорошо, только когда классы линейно разделимы, что в реальных задачах бывает очень редко. Поэтому обычно данная модель используется в качестве baseline.

В завершение изучения логистической регрессии можно добавить, что недостаток с линейной разделимостью классов можно побороть с помощью введения полиномиальных признаков, тем самым снизив смещение модели. Тогда логистическая регрессия вместо разделяющей плоскости будет означать выгнутую разделяющую поверхность сложной структуры.

Однако мы знаем, что с этим трюком стоит быть аккуратным, так как можно получить переобученную модель. Поэтому в комбинации с полиномиальными признаками стоит подобрать наилучший параметр регуляризации.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-3_18.png)

На рисунке выше изображены три различные модели:

* первая — простая логистическая регрессия;
* вторая — логистическая регрессия, обученная на полиномиальных признаках второй степени;
* третья — логистическая регрессия, обученная на полиномиальных признаках десятой степени.

Видно, что первая модель обладает низким качеством и не обобщает общей зависимости (у неё высокое смещение). Третья же, напротив, идеально выделяет каждое наблюдение в правильный класс, но является переобученной и также не отражает общей зависимости (у неё высокий разброс). Оптимальной моделью является вторая, которая не подстраивается под индивидуальные наблюдения и отражает общую зависимость в данных.

    ✍ Ранее мы с вами рассмотрели основы бинарной классификации. Но что делать, когда классов, на которые необходимо разделить данные, больше 2? Например, классификация автомобилей по различным маркам или определение национальности по фотографии и т. д.

В таком случае используется очень простой подход, который называется **«один против всех» (one-vs-over)**.

**Идея** *этого подхода очень простая. Если у нас есть $k$ различных классов ($k>2$), давайте обучим $k$ классификаторов, каждый из которых будет предсказывать вероятности принадлежности каждого объекта к определённому классу.*

Например, у нас есть три класса, обозначенные как 0, 1 и 2. Тогда мы обучаем три классификатора: первый из них учится отличать класс 0 от классов 1 и 2, второй — класс 1 от классов 0 и 2, а третий — класс 2 от классов 1 и 0. Таким образом, класс, на который «заточен» классификатор, мы обозначаем как 1, а остальные классы — как 0.

Когда каждая из трёх моделей сделает предсказание вероятностей для объекта, итоговый классификатор будет выдавать класс, который соответствует самой «уверенной» модели.

Схематично это можно представить следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_4_1.png)

Если мы используем в качестве классификатора логистическую регрессию и количество факторов равно двум ($x_1$ и $x_2$), то можно изобразить тепловую карту вероятностей принадлежности к каждому из классов в каждой точке пространства, а также разделяющие плоскости, которые образуются при пороге вероятности в 0.5.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-3_10.png)

На тепловых картах каждый класс обозначен своим цветом: 0 — зелёным, 1 — жёлтым, 2 — синим. Чем ярче цвет, тем выше вероятность принадлежности к каждому к классу в этой области пространства.

В результате у нас получится три различных пространства вероятностей, что-то вроде трёх параллельных реальностей. Чтобы собрать всё это воедино, мы выбираем в каждой точке пространства максимум из вероятностей. Получим следующую картину:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-3_11.png)

Модель логистической регрессии легко обобщается на случай мультиклассовой классификации. Пусть мы построили несколько разделяющих плоскостей с различными наборами параметров $k$, где $k$ — номер классификатора. То есть имеем $K$ разделяющих плоскостей:

![](./image/ML_3_17.png)

Чтобы преобразовать результат каждой из построенных моделей в вероятности в логистической регрессии, используется функция softmax — многомерный аналог сигмоиды:

![](./image/ML_3_18.png)

Данная функция выдаёт нормированные вероятности, то есть в сумме для всех классов вероятность будет равна 1.

### <center>МУЛЬТИКЛАССОВАЯ КЛАССИФИКАЦИЯ НА PYTHON

Код и результаты находятся в `Мультиклассовая_классификация.ipynb`